In [1]:
import requests
import pandas as pd
import numpy as np
import datetime
import simplekml
import json


In [2]:
URL = "https://climbingaway.fr/en/rock-climbing-areas/world-map-of-rock-climbing-areas/index.php?option=com_cartes_sites&task=getMarkPays"

In [3]:
r = requests.post(URL, data = {'id_pays':16, 'nom_region':'NA'})

In [4]:
data = r.json()
for obj in data['data']:
    print(obj)
    break

{'id': '28', 'nom': 'Fontainebleau (Le Bas Cuvier)', 'type_escalade': 'Bloc-', 'exposition': 'Toutes-', 'public': 'hautniveau-confirme-amateur-debutant-', 'pieddesvoies': 'confortable', 'ref_topo': '-268-1562-148-266-267-192-583-', 'coor_1': '48.447084594124036', 'coor_2': '2.6389431953703024', 'equipement': '', 'topo_num': '0', 'saison': '1-2-3-4-5-10-11-12'}


In [5]:
def request_data(id_pays):
    
    df = pd.DataFrame (columns=['id','nom','type_escalade','exposition','public',
                                'pieddesvoies','ref_topo','coor_1','coor_2','equipement','topo_num','saison'])
    r = requests.post(URL, data = {'id_pays':id_pays, 'nom_region':'NA'})

    data = r.json()
    
    for obj in data['data']:
        climbing_spot = {
            'id':obj['id'],
            'nom':obj['nom'],
            'type_escalade':obj['type_escalade'],
            'exposition':obj['exposition'],
            'public':obj['public'],
            'pieddesvoies':obj['pieddesvoies'],
            'ref_topo':obj['ref_topo'],
            'coor_1':obj['coor_1'],
            'coor_2':obj['coor_2'],
            'equipement':obj['equipement'],
            'topo_num':obj['topo_num'],
            'saison':obj['saison']}
        
        df = df.append(climbing_spot, ignore_index=True)
    return df

In [6]:
df = request_data(16)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138 entries, 0 to 2137
Data columns (total 12 columns):
id               2138 non-null object
nom              2138 non-null object
type_escalade    2138 non-null object
exposition       2138 non-null object
public           2138 non-null object
pieddesvoies     2138 non-null object
ref_topo         2138 non-null object
coor_1           2138 non-null object
coor_2           2138 non-null object
equipement       2138 non-null object
topo_num         2138 non-null object
saison           2138 non-null object
dtypes: object(12)
memory usage: 200.5+ KB


In [8]:
df.head()

,id,nom,type_escalade,exposition,public,pieddesvoies,ref_topo,coor_1,coor_2,equipement,topo_num,saison
0,28,Fontainebleau (Le Bas Cuvier),Bloc-,Toutes-,hautniveau-confirme-amateur-debutant-,confortable,-268-1562-148-266-267-192-583-,48.447084594124036,2.6389431953703024,,0,1-2-3-4-5-10-11-12
1,29,Les Calanques - Sugiton (La Paroi des Toits),Grande voie-Couenne-,SW-S-,hautniveau-confirme-amateur-,correct,-28-,43.213578567634364,5.45468384029391,Sportif-,1,1-2-3-4-5-6-9-10-11-12
2,37,Saint-Bauzille-de-Montmel,Couenne-,SW-S-SE-,confirme-amateur-debutant-,accidenté,-88-,43.77526622417932,3.9485549926757812,Sportif-,0,1-2-3-4-5-6-9-10-11-12
3,38,Buoux,Grande voie-Couenne-,W-SW-S-SE-E-,hautniveau-confirme-amateur-debutant-,accidenté,-79-,43.822627394853725,5.382528305053711,Sportif-,0,3-4-5-6-9-10-11-12
4,39,Tende,Couenne-,SW-,hautniveau-confirme-,correct,-33-,44.102554246308976,7.588924169540405,Sportif-,0,4-5-6-9-10-11


In [ ]:
kml = simplekml.Kml()
for index, row in df.iterrows():
    pnt = kml.newpoint(name="",
                 description="Name: {} - Type: {}".format(row.nom, row.type_escalade),
                 coords=[(row.coor_2, row.coor_1)])
    pnt.style.iconstyle.icon.href = "http://maps.google.com/mapfiles/kml/paddle/blu-circle.png"
kml.save("data/climbing_spots.kml")